# Distributed PyTorch training With Azure ML

<b> Objective : To create a multiclass classification model with PyTorch on Azure ML</b>

In [1]:
%%capture

!mkdir ./newsDataset
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/train.csv -P ./newsDataset
!wget https://raw.githubusercontent.com/mhjabreel/CharCnn_Keras/master/data/ag_news_csv/test.csv -P ./newsDataset

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()
dstore = ws.get_default_datastore()

dstore.upload('./newsDataset', '/news-dataset', show_progress = True)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 2 files
Uploading ./newsDataset/test.csv
Uploaded ./newsDataset/test.csv, 1 files out of an estimated total of 2
Uploading ./newsDataset/train.csv
Uploaded ./newsDataset/train.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_7370071f1fc144e5b73b4f8df75667d6

In [5]:
#Registering the dataset
from azureml.core import Dataset

trainDataset =  Dataset.Tabular.from_delimited_files(dstore.path('news-dataset/train.csv'))
testDataset =  Dataset.Tabular.from_delimited_files(dstore.path('news-dataset/test.csv'))

trainDataset.register(workspace=ws, name='news-train-dataset', description='news train dataset')
testDataset.register(workspace=ws, name='news-test-dataset', description='news test dataset')

{
  "source": [
    "('workspaceblobstore', 'news-dataset/test.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "3aa5e6ad-721b-4d69-8110-1f9161f71036",
    "name": "news-test-dataset",
    "version": 1,
    "description": "news test dataset",
    "workspace": "Workspace.create(name='mlops-san', subscription_id='0d1442c1-d386-4505-9abe-0bedfd63701e', resource_group='mlops-san')"
  }
}

# PyTorch Script - Model with training and evaluation script

In [12]:
from azureml.core import Workspace, Dataset
import torchtext
from torchtext.data.utils import get_tokenizer, ngrams_iterator

ws = Workspace.from_config() # create a hidden folder called .azureml and place the config.json there

train_dataset = Dataset.get_by_name(ws, name='news-train-dataset')
train_df = train_dataset.to_pandas_dataframe()

test_dataset = Dataset.get_by_name(ws, name='news-test-dataset')
test_df = test_dataset.to_pandas_dataframe()

train_text, train_label = train_df.iloc[:, 2].map(lambda x : x.lower()), train_df.iloc[:, 0].map(lambda x : int(x) - 1)
test_text, test_label = test_df.iloc[:, 2].map(lambda x : x.lower()), test_df.iloc[:, 0].map(lambda x : int(x) - 1)

def tokenizer():
    pass

ModuleNotFoundError: No module named 'torchtext'

In [10]:
from torchtext.data.utils import get_tokenizer, ngrams_iterator

tokenizer = get_tokenizer("basic_english")

tokens = tokenizer("You can now install TorchText using pip!")

ModuleNotFoundError: No module named 'torchtext'

In [ ]:
import torch
import torch.nn as nn
from torch.utils import data


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')